### Libraries

In [145]:
# import required libraries
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch
import matplotlib_inline
from matplotlib import cm
import matplotlib_inline
import seaborn as sns
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

### Convert event data from .json match file into .csv

In [958]:
# extract all match names in directory as a list 
files = os.listdir('matches_2021_json')

# avoid last file 'players.json' indexed as '-1'
for i in files[:-1]:
    with open('matches_2021_json\\' + i, 'r') as f:
        data = json.load(f)
   
    # convert list of dictionaries into data frame 
    events = pd.DataFrame(data['events'])
    
    # generate 'team_status' column which indicates whether a team plays 'home' or 'away'
    events['team_status'] = events['team'].apply(lambda x: 'home' if x == data['homeTeam'] else 'away')
    
    # reset indices
    events = events.reset_index(drop = True)
    

    # since only single match has less than 23 columns which is '397511822166547-Halmstad-DegerforsIF'
    if len(events.columns) < 23:
        # create missing columns with 'nan' values to have equal # of columns in all .csv files
        events[['attack_type', 'corner_outcome', 'foot_used', 'one_touch', 'goal_mouth']] = \
        pd.DataFrame([np.repeat(np.nan, 5)], index = events.index)
    
    # preferred rearrangement of columns
    col_rearr = ['external_id','team', 'team_status', 'player', 'next_player', 'xpos', 'ypos', 'xdest', 'ydest', 'start_time', 
                     'end_time','action', 'xg', 'xt', 'xp','game_time', 'header', 'penalty', 'throw_in', 'attack_type', 
                    'corner_outcome', 'foot_used', 'one_touch', 'goal_mouth']
    # rearrange columns
    events = events[col_rearr]
    
    # replace all None values with NaN (missing value)
    events.fillna(value = np.nan, inplace = True)
    
    
    # make certain column values lowercase
    events[['action', 'attack_type', 'corner_outcome', 'foot_used', 'goal_mouth']] = \
    events[['action', 'attack_type', 'corner_outcome', 'foot_used', 'goal_mouth']].astype(str).apply(lambda x: x.str.lower())
    
    # save dataframe 
    file_name = i.split('.')[0]
    events.to_csv('matches_2021_csv\\' + file_name + '.csv', index = False)

### Minutes played by players data

In [669]:
# load info 'how much players played in each game'
with open('matches_2021_json\\allsvenskan, 2021-players.json', 'r') as f:
    data2 = json.load(f)

# aggregate results into data frame
playing_time = pd.DataFrame()
for i in data2:
    single_col = {'player_name':i['name'], 'team_name':i['team'],
                  'minutes_played':sum(i['minutes']['62163887d0736a3c7964ff4b'].values())}
    playing_time = pd.concat([playing_time, pd.DataFrame([single_col])])

playing_time = playing_time.reset_index(drop = True)

In [670]:
playing_time.head()

,player_name,team_name,minutes_played
0,Christopher Mc Vey,IF Elfsborg,2307
1,Tashreeq Matthews,Varbergs BoIS FC,1969
2,Ali Youssef,BK Häcken,389
3,Mikael Lustig,AIK,2491
4,Davor Blazevic,Hammarby,96


In [ ]:
# Incorrect minutes data for players. Examples are below:
# Justin Salmon, Degerfors
# Ali Youssef, Hacken

In [ ]:
# quick links for check:
# https://fbref.com/en/comps/29/11002/2021-Allsvenskan-Stats
# https://fbref.com/en/comps/29/11002/stats/2021-Allsvenskan-Stats#all_stats_standard

### Refining FBref data
`Match results` with all match scores and `player_stats` with minutes played by players

In [67]:
# load fbfref data
match_results = pd.read_csv("match_results.csv")
# create column indices to be removed
rm_col_ind = np.r_[0:5, 6, -5:0]
# remove columns
match_results = match_results.drop(columns = match_results.columns[rm_col_ind], axis = 1)
# make all column names lowercase
match_results = match_results.rename(columns = str.lower)
# save refined .csv file
match_results.to_csv('match_results.csv', index = False)

In [49]:
# load fbref data 
player_stats = pd.read_csv('player_stats.csv')
# create column indices to be removed
rm_col_ind = np.r_[0, 2, -1]
# remove columns
player_stats = player_stats.drop(columns = player_stats.columns[rm_col_ind], axis = 1)
# make all column names lowercase
player_stats = player_stats.rename(columns = str.lower)
player_stats.to_csv('player_stats.csv', index = False)

### Save all match events into a single dataframe

In [671]:
csv_files = os.listdir('matches_2021_csv/')

all_match_events = pd.DataFrame()
for i in csv_files:
    df = pd.read_csv('matches_2021_csv/'+ i)
    all_match_events = pd.concat([all_match_events, df])
all_match_events.to_csv('all_match_events.csv', index = False)    

In [672]:
all_match_events.shape

(409713, 23)

In [674]:
all_match_events.head()

,external_id,team,player,next_player,xpos,ypos,xdest,ydest,start_time,end_time,...,xp,game_time,header,penalty,throw_in,attack_type,corner_outcome,foot_used,one_touch,goal_mouth
0,0,BK Häcken,Tobias Heintz,Alexander Faltsetas,51,50,37.0,56.0,0,3,...,0.882111,1,False,False,False,NaN,NaN,NaN,NaN,NaN
1,1,BK Häcken,Alexander Faltsetas,Godswill Ekpolo,37,52,40.0,9.0,1,4,...,0.772046,1,False,False,False,NaN,NaN,NaN,NaN,NaN
2,2,BK Häcken,Godswill Ekpolo,Joona Toivio,41,7,27.0,25.0,7,10,...,0.906022,1,False,False,False,NaN,NaN,NaN,NaN,NaN
3,3,BK Häcken,Joona Toivio,Johan Hammar,25,29,25.0,68.0,10,13,...,0.895334,1,False,False,False,NaN,NaN,NaN,NaN,NaN
4,4,BK Häcken,Johan Hammar,Joona Toivio,31,63,31.0,33.0,15,18,...,0.867001,1,False,False,False,NaN,NaN,NaN,NaN,NaN


In [ ]:
# https://stackoverflow.com/questions/26139423/plot-different-color-for-different-categorical-levels-using-matplotlib

### Derive xG results per match

In [1113]:
json_files = os.listdir('matches_2021_json/')

In [1114]:
# Create data frame with results of all matches
xg_results = pd.DataFrame()

# avoid last file 'players.json'
for i in json_files[:-1]:
    with open('matches_2021_json/' + i, 'r') as f:
        data = json.load(f)

    single_col = {'home':data['homeTeam'], 'away':data['visitingTeam'], 
              'home_xg':json.loads(data['xG'])[0], 'away_xg':json.loads(data['xG'])[1]}

    xg_results = pd.concat([xg_results, pd.DataFrame([single_col])])
xg_results = xg_results.reset_index(drop = True)

In [1115]:
xg_results.head()

,home,away,home_xg,away_xg
0,BK Häcken,Örebro,3.136022,0.946330
1,Kalmar FF,Östersund,0.811020,0.156506
2,Degerfors IF,IFK Göteborg,0.652445,0.908759
3,IF Elfsborg,Varbergs BoIS FC,0.381496,1.120826
4,Örebro,AIK,1.274922,3.121992


In [272]:
# sanity check of xG results from .csv file
csv_files = os.listdir('matches_2021_csv/')
df = pd.read_csv('matches_2021_csv/' + csv_files[0])
df[(df['action'] == 'Shot') | (df['action'] == 'Goal')][['action', 'team', 'xg']].groupby(by = 'team').sum()

,xg
team,
BK Häcken,3.136022
Örebro,0.946330


### Derive xG of penalties per match

In [1029]:
# list all .csv files for matches
files = os.listdir('matches_2021_csv/')

match_penalties = pd.DataFrame()

for i in files:
    
    df = pd.read_csv('matches_2021_csv/' + i)
    
    # check for penalty condition in a game 
    if df[df['penalty'] == True].empty:
        # since there is no condition assign '0' for penalty xG for each team
        new_df = pd.DataFrame([{
            "home":df[df['team_status'] == 'home']['team'].iloc[0],
            "home_pen_xg": 0 ,
            "away":df[df['team_status'] == 'away']['team'].iloc[0],
            "away_pen_xg": 0 
        }])    

    else:
        # create a conditional dataframe which only includes teams that scored a penalty goal 
        new_df = df[(df['penalty'] == True) & (df['action'] == 'goal')]
        
        # assign all grouped xG from penalties to each team
        # if one of teams did not score penalty then assign '0' to its xG from penalty
        # name of each team is taken from original 'df' dataframe using 'team_status' column
        new_df = pd.DataFrame([{
              "home" : df[df['team_status'] == 'home']['team'].iloc[0],
              "home_pen_xg" : 0 if new_df[new_df['team_status'] == 'home'].empty else 
                             new_df[new_df['team_status'] == 'home'].groupby(by = 'team').sum()['xg'][0],
            
             "away" : df[df['team_status'] == 'away']['team'].iloc[0],
             "away_pen_xg" : 0 if new_df[new_df['team_status'] == 'away'].empty else 
                             new_df[new_df['team_status'] == 'away'].groupby(by = 'team').sum()['xg'][0]
                            }])

    match_penalties = pd.concat([match_penalties, new_df])

match_penalties = match_penalties.reset_index(drop = True)
match_penalties.to_csv('match_penalties.csv', index = False)
# filter out games without penalties
only_pens = match_penalties[(match_penalties['home_pen_xg'] != 0) | (match_penalties['away_pen_xg'] != 0)].reset_index(drop = True)

In [1030]:
only_pens

,home,home_pen_xg,away,away_pen_xg
0,BK Häcken,0.816423,Örebro,0.000000
1,Degerfors IF,1.652965,BK Häcken,0.000000
2,IK Sirius FK,0.000000,Malmö FF,0.640458
3,IK Sirius FK,0.743797,BK Häcken,0.000000
4,Degerfors IF,0.000000,Hammarby,0.691192
5,IF Elfsborg,0.752708,Örebro,0.000000
6,Kalmar FF,0.815195,Varbergs BoIS FC,0.000000
7,Malmö FF,0.725091,Hammarby,0.000000
8,Mjällby AIF,0.616974,IF Elfsborg,0.000000
9,AIK,0.773842,IF Elfsborg,0.000000


In [1020]:
# number of penalties scored by each team
penalties_by_team = all_match_events[(all_match_events['penalty'] == True) & (all_match_events['action'] == 'goal')]

In [1021]:
penalties_by_team[(penalties_by_team['action'] == 'goal')][['team', 'penalty', 'xg']].groupby(by = 'team').sum()

,penalty,xg
team,,
AIK,1,0.773842
BK Häcken,5,3.925681
Degerfors IF,4,3.361853
Djurgården,2,1.500876
Halmstad,2,1.532405
Hammarby,3,2.123771
IF Elfsborg,3,2.273832
IFK Göteborg,3,2.187699
IFK Norrköping FK,1,0.870289


In [ ]:
# Notes: Why there are different xG for penalties ?
#

### Replace team names in fbref data with team names in playmakerAI data

In [1117]:
xg_names = xg_results['home'].unique()
xg_results['home'].unique()

array(['BK Häcken', 'Kalmar FF', 'Degerfors IF', 'IF Elfsborg', 'Örebro',
       'Malmö FF', 'Östersund', 'IK Sirius FK', 'Hammarby',
       'IFK Norrköping FK', 'Halmstad', 'IFK Göteborg', 'AIK',
       'Mjällby AIF', 'Djurgården', 'Varbergs BoIS FC'], dtype=object)

In [1118]:
match_results = pd.read_csv('match_results.csv')
match_names = match_results['home'].unique()
match_results['home'].unique()

array(['Malmö', 'Örebro', 'Halmstad', 'Mjällby', 'Elfsborg', 'Norrköping',
       'AIK Stockholm', 'Kalmar', 'Hammarby', 'Degerfors', 'Östersund',
       'Häcken', 'Djurgården', 'Sirius', 'Göteborg', 'Varberg'],
      dtype=object)

In [1119]:
xg_ls = []
for i in xg_names:
    xg_ls.append(i.split(' '))

xg_ls

[['BK', 'Häcken'],
 ['Kalmar', 'FF'],
 ['Degerfors', 'IF'],
 ['IF', 'Elfsborg'],
 ['Örebro'],
 ['Malmö', 'FF'],
 ['Östersund'],
 ['IK', 'Sirius', 'FK'],
 ['Hammarby'],
 ['IFK', 'Norrköping', 'FK'],
 ['Halmstad'],
 ['IFK', 'Göteborg'],
 ['AIK'],
 ['Mjällby', 'AIF'],
 ['Djurgården'],
 ['Varbergs', 'BoIS', 'FC']]

In [1120]:
names_dict = {}
for i in xg_ls:
    for j in i:
        if j in match_names:
            full_name = ' '.join(i)
            short_name = list(match_names[match_names == j])[0]
            names_dict[full_name] = short_name

# special cases 'AIK Stockholm' and 'Varbergs'        
names_dict['AIK'] = 'AIK Stockholm'
names_dict['Varbergs BoIS FC'] = 'Varberg' 

# reverse keys and values in
names_dict = dict((value, key) for key, value in names_dict.items())
names_dict

{'Häcken': 'BK Häcken',
 'Kalmar': 'Kalmar FF',
 'Degerfors': 'Degerfors IF',
 'Elfsborg': 'IF Elfsborg',
 'Örebro': 'Örebro',
 'Malmö': 'Malmö FF',
 'Östersund': 'Östersund',
 'Sirius': 'IK Sirius FK',
 'Hammarby': 'Hammarby',
 'Norrköping': 'IFK Norrköping FK',
 'Halmstad': 'Halmstad',
 'Göteborg': 'IFK Göteborg',
 'Mjällby': 'Mjällby AIF',
 'Djurgården': 'Djurgården',
 'AIK Stockholm': 'AIK',
 'Varberg': 'Varbergs BoIS FC'}

In [1121]:
# replace team names in fbref data with original names from playmaker AI
match_results['home'] = match_results['home'].apply(lambda x: names_dict[x])
match_results['away'] = match_results['away'].apply(lambda x: names_dict[x])
match_results.head()

,wk,date,time,home,homegoals,away,awaygoals
0,1,2021-04-10,15:00,Malmö FF,3,Hammarby,2
1,1,2021-04-10,17:30,Örebro,0,IFK Göteborg,0
2,1,2021-04-11,15:00,Halmstad,1,BK Häcken,0
3,1,2021-04-11,15:00,Mjällby AIF,0,Varbergs BoIS FC,0
4,1,2021-04-11,17:30,IF Elfsborg,0,Djurgården,2


In [1124]:
# save refined .csv file
match_results.to_csv('match_results.csv', index = False)

### Merge FBref and PlaymakerAI Data
`Match scores` from FBref and `match xg scores` from PlaymakerAI

In [1125]:
# PlaymakerAI data with xg scores per match
xg_results.head()

,home,away,home_xg,away_xg
0,BK Häcken,Örebro,3.136022,0.946330
1,Kalmar FF,Östersund,0.811020,0.156506
2,Degerfors IF,IFK Göteborg,0.652445,0.908759
3,IF Elfsborg,Varbergs BoIS FC,0.381496,1.120826
4,Örebro,AIK,1.274922,3.121992


In [1031]:
# PlaymakerAI data with xG from penalties per match
match_penalties.head()

,home,home_pen_xg,away,away_pen_xg
0,BK Häcken,0.816423,Örebro,0.0
1,Kalmar FF,0.000000,Östersund,0.0
2,Degerfors IF,0.000000,IFK Göteborg,0.0
3,IF Elfsborg,0.000000,Varbergs BoIS FC,0.0
4,Örebro,0.000000,AIK,0.0


In [1126]:
# Merge PlaymakerAI data
merged_xg_results = xg_results.merge(right = match_penalties, on = ['home', 'away'])
merged_xg_results.head()

,home,away,home_xg,away_xg,home_pen_xg,away_pen_xg
0,BK Häcken,Örebro,3.136022,0.946330,0.816423,0.0
1,Kalmar FF,Östersund,0.811020,0.156506,0.000000,0.0
2,Degerfors IF,IFK Göteborg,0.652445,0.908759,0.000000,0.0
3,IF Elfsborg,Varbergs BoIS FC,0.381496,1.120826,0.000000,0.0
4,Örebro,AIK,1.274922,3.121992,0.000000,0.0


In [1127]:
merged_xg_results.shape

(240, 6)

In [1128]:
match_results.shape

(240, 7)

In [1131]:
# merge two dataframes on column names 'home' and 'away'
merged_results = match_results.merge(merged_xg_results, on = ['home', 'away'])
merged_results

,wk,date,time,home,homegoals,away,awaygoals,home_xg,away_xg,home_pen_xg,away_pen_xg
0,1,2021-04-10,15:00,Malmö FF,3,Hammarby,2,3.081973,0.284890,0.725091,0.000000
1,1,2021-04-10,17:30,Örebro,0,IFK Göteborg,0,0.630045,0.747871,0.000000,0.000000
2,1,2021-04-11,15:00,Halmstad,1,BK Häcken,0,1.014930,0.405163,0.000000,0.000000
3,1,2021-04-11,15:00,Mjällby AIF,0,Varbergs BoIS FC,0,1.517195,0.707990,0.000000,0.000000
4,1,2021-04-11,17:30,IF Elfsborg,0,Djurgården,2,0.841527,0.221132,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
235,30,2021-12-04,15:00,Varbergs BoIS FC,0,Mjällby AIF,3,1.136151,1.983585,0.000000,0.699833
236,30,2021-12-04,15:00,IFK Norrköping FK,1,IFK Göteborg,2,0.936531,1.369556,0.000000,0.000000
237,30,2021-12-04,15:00,Hammarby,5,Kalmar FF,3,6.646094,1.709884,0.000000,0.000000
238,30,2021-12-04,15:00,Malmö FF,0,Halmstad,0,1.232243,0.073184,0.000000,0.000000


In [1132]:
# derive non-penalty xg for 'home' and 'away' teams
merged_results['home_np_xg'] = merged_results['home_xg'] - merged_results['home_pen_xg']
merged_results['away_np_xg'] = merged_results['away_xg'] - merged_results['away_pen_xg']
merged_results

,wk,date,time,home,homegoals,away,awaygoals,home_xg,away_xg,home_pen_xg,away_pen_xg,home_np_xg,away_np_xg
0,1,2021-04-10,15:00,Malmö FF,3,Hammarby,2,3.081973,0.284890,0.725091,0.000000,2.356882,0.284890
1,1,2021-04-10,17:30,Örebro,0,IFK Göteborg,0,0.630045,0.747871,0.000000,0.000000,0.630045,0.747871
2,1,2021-04-11,15:00,Halmstad,1,BK Häcken,0,1.014930,0.405163,0.000000,0.000000,1.014930,0.405163
3,1,2021-04-11,15:00,Mjällby AIF,0,Varbergs BoIS FC,0,1.517195,0.707990,0.000000,0.000000,1.517195,0.707990
4,1,2021-04-11,17:30,IF Elfsborg,0,Djurgården,2,0.841527,0.221132,0.000000,0.000000,0.841527,0.221132
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,30,2021-12-04,15:00,Varbergs BoIS FC,0,Mjällby AIF,3,1.136151,1.983585,0.000000,0.699833,1.136151,1.283752
236,30,2021-12-04,15:00,IFK Norrköping FK,1,IFK Göteborg,2,0.936531,1.369556,0.000000,0.000000,0.936531,1.369556
237,30,2021-12-04,15:00,Hammarby,5,Kalmar FF,3,6.646094,1.709884,0.000000,0.000000,6.646094,1.709884
238,30,2021-12-04,15:00,Malmö FF,0,Halmstad,0,1.232243,0.073184,0.000000,0.000000,1.232243,0.073184


In [1134]:
# save .csv file
merged_results.to_csv('merged_match_results.csv', index = False)